In [1]:
import os
from tqdm.autonotebook import tqdm, trange

import torch
import torch.nn as nn

import torch.optim as optim
from torch.optim import lr_scheduler

import numpy as np
import torchvision
import pandas as pd
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
from matplotlib.cbook import file_requires_unicode
from PIL import Image
from sklearn.model_selection import train_test_split
import urllib.request
from torch.utils.data import DataLoader
import os.path
from os import path
import requests
from io import BytesIO

<ipython-input-1-ef16e35810c3>:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Dataset from https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-small

In [20]:
styles = pd.read_csv("/content/drive/MyDrive/styles.csv",on_bad_lines='skip')
images = pd.read_csv("/content/drive/MyDrive/images.csv")


In [21]:
styles.shape, images.shape

((44424, 10), (44446, 2))

In [22]:
#%rm -rf /content/sample_data/data/train
#%rm -rf /content/sample_data/data/


In [23]:
import time

Downloading data:

In [24]:
!unzip /content/sample_data/dataimages.csv.zip
!unzip /content/sample_data/styles.csv.zip

unzip:  cannot find or open /content/sample_data/dataimages.csv.zip, /content/sample_data/dataimages.csv.zip.zip or /content/sample_data/dataimages.csv.zip.ZIP.
unzip:  cannot find or open /content/sample_data/styles.csv.zip, /content/sample_data/styles.csv.zip.zip or /content/sample_data/styles.csv.zip.ZIP.


In [25]:
#images = pd.read_csv("/content/sample_data/images.csv")
#images.head()

In [26]:
styles = pd.read_csv("/content/sample_data/styles.csv",on_bad_lines='skip')
styles.shape, images.shape

((44424, 10), (44446, 2))

In [27]:
counts = styles["articleType"].value_counts()
styles.shape

(44424, 10)

Delete all the things that are less than 100 in the data:

In [64]:
counts = styles["articleType"].value_counts()
new_style = styles["articleType"].map(counts)
styles1 = styles[new_style > 2000]
styles1 = styles1[["id", "articleType"]]
styles1 = pd.DataFrame(styles1)
dataset = styles1
dataset.shape
dataset["articleType"].unique()

array(['Shirts', 'Watches', 'Tshirts', 'Casual Shoes', 'Sports Shoes'],
      dtype=object)

In [65]:
dataset

,id,articleType
0,15970,Shirts
2,59263,Watches
4,53759,Tshirts
5,1855,Tshirts
6,30805,Shirts
...,...,...
44415,13496,Tshirts
44417,12544,Tshirts
44419,17036,Casual Shoes
44421,18842,Tshirts


Data processing:

In [66]:
images = images.rename(columns={"filename" : "id"})
images

,id,link
0,15970,http://assets.myntassets.com/v1/images/style/p...
1,39386,http://assets.myntassets.com/v1/images/style/p...
2,59263,http://assets.myntassets.com/v1/images/style/p...
3,21379,http://assets.myntassets.com/v1/images/style/p...
4,53759,http://assets.myntassets.com/v1/images/style/p...
...,...,...
44441,17036,http://assets.myntassets.com/v1/images/style/p...
44442,6461,http://assets.myntassets.com/v1/images/style/p...
44443,18842,http://assets.myntassets.com/v1/images/style/p...
44444,46694,http://assets.myntassets.com/v1/images/style/p...


In [75]:
images["id"] = images["id"].str.replace('.jpg', '', regex=True)

AttributeError: ignored

In [76]:
images = images.rename(columns={"filename" : "id"})

In [77]:
images["id"] = images["id"].astype("int")

In [78]:
images["id"].isna().sum()

0

In [79]:
images = images.rename(columns={"filename" : "id"})
images["id"] = images["id"].astype("int64")
images["id"].info
images.shape

(44446, 2)

Make styles.csv and images.csv the same size:

In [80]:
a = {}
for i in range(images.shape[0]):
  a[images["id"][i]] = images["link"][i]
a[15970]

'http://assets.myntassets.com/v1/images/style/properties/7a5b82d1372a7a5c6de67ae7a314fd91_images.jpg'

In [81]:

dataset["id"].tail(1)

44423    51623
Name: id, dtype: int64

In [82]:
dataset["link"] = dataset["id"].map(a)
dataset.isna().sum()

id             0
articleType    0
link           0
dtype: int64

In [83]:
dataset = dataset.dropna()
dataset.isna().sum()


id             0
articleType    0
link           0
dtype: int64

Split into train and test:

In [84]:
train_set, val_set, y_train, y_val = train_test_split(dataset[["id", "link", "articleType"]], dataset[["id", "articleType"]] , test_size=0.20, random_state=42)

In [85]:
len(train_set), len(val_set)

(14165, 3542)

In [86]:
dataset.head()

,id,articleType,link
0,15970,Shirts,http://assets.myntassets.com/v1/images/style/p...
2,59263,Watches,http://assets.myntassets.com/v1/images/style/p...
4,53759,Tshirts,http://assets.myntassets.com/v1/images/style/p...
5,1855,Tshirts,http://assets.myntassets.com/v1/images/style/p...
6,30805,Shirts,http://assets.myntassets.com/v1/images/style/p...


New folders for data:

In [43]:
!mkdir /content/sample_data/data
!mkdir /content/sample_data/data/train
!mkdir /content/sample_data/data/val

mkdir: cannot create directory ‘/content/sample_data/data’: File exists
mkdir: cannot create directory ‘/content/sample_data/data/train’: File exists
mkdir: cannot create directory ‘/content/sample_data/data/val’: File exists


In [44]:
#%rm -rf /content/sample_data/data/val


function for loading image from url and resizing into one shape for all images:


(Also collects them in special folders)

In [87]:
def load_image(link, phase, article, id):
    transform = torchvision.transforms.Resize((250, 250))

    response = requests.get(link)
    image = Image.open(BytesIO(response.content))
    image=image.convert('RGB')
    image = transform(image)
    file_path = "/content/sample_data/data/%s/%s/%d.jpg" % (phase, article, id)
    folder_path = "/content/sample_data/data/%s/%s" % (phase, article)

    if os.path.exists(folder_path):
      return image.save(file_path)
    else:
      os.mkdir(folder_path)
      return image.save(file_path)


In [88]:
def loading(dataset, n, phase):
  for i in range(n):
    load_image(dataset["link"].values[i], phase, dataset["articleType"].values[i], dataset["id"].values[i])


In [97]:
%rm -rf /content/sample_data/data/train
!mkdir /content/sample_data/data/train

In [98]:
%rm -rf /content/sample_data/data/val
!mkdir /content/sample_data/data/val

collects them into one folder (train):

In [ ]:
N = 150
for i in range(N):
  if i % 10 == 0:
    print(i)
  loading(train_set, i, "train")

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140


collects them into one folder (validation set):

In [120]:
N = 20
for i in range(N):
  loading(val_set, i, "val")

Transform images to tensors and normalize them:

In [121]:

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '/content/sample_data/data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                              shuffle=True)
               for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

use_gpu = torch.cuda.is_available()

In [122]:
dataloaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7c0b25b5afe0>,
 'val': <torch.utils.data.dataloader.DataLoader at 0x7c0b25b5b700>}

Model training:

In [124]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    losses = {'train': [], "val": []}

    pbar = trange(num_epochs, desc="Epoch:")

    for epoch in pbar:

        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0


            for data in tqdm(dataloaders[phase], leave=False, desc=f"{phase} iter:"):
                inputs, labels = data

                if use_gpu:
                    inputs = inputs.cuda()
                    labels = labels.cuda()
                else:
                    inputs, labels = inputs, labels

                if phase=="train":
                    optimizer.zero_grad()

                if phase == "eval":
                    with torch.no_grad():
                        outputs = model(inputs)
                else:
                    outputs = model(inputs)
                preds = torch.argmax(outputs, -1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                running_loss += loss.item()
                running_corrects += int(torch.sum(preds == labels.data))

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]


            losses[phase].append(epoch_loss)

            pbar.set_description('{} Loss: {:.4f} Acc: {:.4f}'.format(
                                    phase, epoch_loss, epoch_acc
                                ))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))


    model.load_state_dict(best_model_wts)
    return model, losses

In [125]:
model = models.alexnet(pretrained=True)

In [126]:
loss_fn = nn.CrossEntropyLoss()

optimizer_ft = optim.Adam(model.parameters(), lr=1e-4)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.1)
if torch.cuda.is_available():
    model.cuda()




In [127]:
model, losses = train_model(model, loss_fn, optimizer_ft, exp_lr_scheduler, num_epochs=10)

Epoch::   0%|          | 0/10 [00:00<?, ?it/s]

train iter::   0%|          | 0/38 [00:00<?, ?it/s]

val iter::   0%|          | 0/5 [00:00<?, ?it/s]

train iter::   0%|          | 0/38 [00:00<?, ?it/s]

val iter::   0%|          | 0/5 [00:00<?, ?it/s]

train iter::   0%|          | 0/38 [00:00<?, ?it/s]

val iter::   0%|          | 0/5 [00:00<?, ?it/s]

train iter::   0%|          | 0/38 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [128]:
model_extractor = models.alexnet(pretrained=True)

In [129]:
for param in model_extractor.parameters():
    param.requires_grad = False

num_features = 9216
model_extractor.classifier = nn.Linear(num_features, 17)


if use_gpu:
    model_extractor = model_extractor.cuda()

loss_fn = nn.CrossEntropyLoss()

optimizer = optim.Adam(model_extractor.classifier.parameters(), lr=1e-4)


exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [130]:
%%time
model_extractor,losses = train_model(model_extractor, loss_fn, optimizer, exp_lr_scheduler, num_epochs=10)

Epoch::   0%|          | 0/10 [00:00<?, ?it/s]

train iter::   0%|          | 0/38 [00:00<?, ?it/s]

val iter::   0%|          | 0/5 [00:00<?, ?it/s]

train iter::   0%|          | 0/38 [00:00<?, ?it/s]

val iter::   0%|          | 0/5 [00:00<?, ?it/s]

...

train iter::   0%|          | 0/38 [00:00<?, ?it/s]

val iter::   0%|          | 0/5 [00:00<?, ?it/s]

train iter::   0%|          | 0/38 [00:00<?, ?it/s]

val iter::   0%|          | 0/5 [00:00<?, ?it/s]

Training complete in 1m 18s
Best val Acc: 0.947368
CPU times: user 1min 12s, sys: 425 ms, total: 1min 12s
Wall time: 1min 17s


In [62]:
def evaluate(model):
    model.eval()

    runninig_correct = 0
    for data in dataloaders['val']:
        inputs, labels = data

        if use_gpu:
            inputs = inputs.cuda()
            labels = labels.cuda()

        output = model(inputs)
        _, predicted = torch.max(output, 1)

        runninig_correct += int(torch.sum(predicted == labels))

    return runninig_correct / dataset_sizes['val']

In [ ]:
def evaluate(model):
    model.eval()

    runninig_correct = 0
    for data in dataloaders['val']:
        # получаем картинки и метки
        inputs, labels = data

        # переносим на gpu, если возможно
        if use_gpu:
            inputs = inputs.cuda()
            labels = labels.cuda()

        # forard pass
        output = model(inputs)
        _, predicted = torch.max(output, 1)

        runninig_correct += int(torch.sum(predicted == labels))

    return runninig_correct / dataset_sizes['val']